# Experiment 10b: 2D Parameter Sweep (α × σ)

**Phase 4 - Mapping the Recovery-Capable Parameter Region**

## Background

Experiment 10 revealed that varying recovery α alone (1.1 to 2.0) produced uniformly low recovery (~1.3%) across all values. This suggests that **noise amplitude σ may be more important than noise type α** for enabling recovery.

## Revised Hypothesis

Recovery requires BOTH:
1. Heavy-tailed noise (low α) for extreme jumps
2. Sufficient noise amplitude (higher σ) to overcome the barrier

## This Experiment

2D sweep across recovery (α, σ) space to:
1. Map recovery fraction as a function of both parameters
2. Identify the recovery-capable region
3. Determine if there's an α × σ interaction effect
4. Find minimum intervention (σ) needed for recovery at each α

| Parameter | Values |
|-----------|--------|
| Recovery α | [1.2, 1.4, 1.6, 1.8, 2.0] (5 values) |
| Recovery σ | [0.04, 0.06, 0.08, 0.10, 0.12] (5 values) |
| Grid points | 25 combinations |
| Ensemble runs | 20 per combination |
| **Total simulations** | 500 |

## Dask Optimizations Applied

Based on Experiment 10 learnings:
- Pre-scatter network to all workers (broadcast)
- Batch tasks by parameter combination
- Use `client.map()` for homogeneous task submission

## 1. Setup and Imports

In [ ]:
import sys
sys.path.insert(0, '/opt/research-local/src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
from pathlib import Path
from netCDF4 import Dataset
from itertools import product
from dask.distributed import as_completed

# Core energy-constrained module
from energy_constrained import (
    EnergyConstrainedNetwork,
    EnergyConstrainedCusp,
    GradientDrivenCoupling,
    run_two_phase_experiment,
    get_dask_client
)

print("Imports successful!")
print(f"NumPy version: {np.__version__}")

In [ ]:
# Connect to Dask cluster
client = get_dask_client()
print(f"Connected to: {client.scheduler_info()['address']}")
print(f"Workers: {len(client.scheduler_info()['workers'])}")
print(f"Total threads: {sum(w['nthreads'] for w in client.scheduler_info()['workers'].values())}")

## 2. Load Amazon Moisture Recycling Data

In [ ]:
DATA_PATH = Path('/opt/research-local/data/amazon/amazon_adaptation_model/average_network/era5_new_network_data')

def load_amazon_data(year=2003, months=[7, 8, 9]):
    """Load and average Amazon moisture recycling data for specified months."""
    all_rain = []
    all_evap = []
    all_network = []
    
    for month in months:
        file_path = DATA_PATH / f'1deg_{year}_{month:02d}.nc'
        if file_path.exists():
            with Dataset(file_path, 'r') as ds:
                all_rain.append(ds.variables['rain'][:])
                all_evap.append(ds.variables['evap'][:])
                all_network.append(ds.variables['network'][:])
    
    return {
        'rain': np.mean(all_rain, axis=0),
        'evap': np.mean(all_evap, axis=0),
        'network': np.mean(all_network, axis=0),
        'n_cells': len(all_rain[0])
    }

# Load 2003 (normal year) data
amazon_data = load_amazon_data(year=2003)
print(f"Loaded Amazon data: {amazon_data['n_cells']} cells")
print(f"Network shape: {amazon_data['network'].shape}")

## 3. Experiment Configuration

In [ ]:
# Experiment 10b: 2D Alpha-Sigma Sweep Configuration

SWEEP_CONFIG = {
    # 2D sweep parameters
    'alpha_values': np.array([1.2, 1.4, 1.6, 1.8, 2.0]),  # 5 values
    'sigma_values': np.array([0.04, 0.06, 0.08, 0.10, 0.12]),  # 5 values
    'n_runs_per_combo': 20,
    
    # Network parameters
    'n_cells': 50,
    'min_flow': 1.0,
    'barrier_height': 0.2,  # Option D from Exp 9
    
    # Two-phase simulation parameters
    'cascade_duration': 200,
    'recovery_duration': 800,
    'dt': 0.5,
    'cascade_sigma': 0.06,
    'cascade_alpha': 1.5,  # Fixed Lévy for cascade
    
    # Seeds
    'base_seed': 42,
}

# Calculate grid
n_alpha = len(SWEEP_CONFIG['alpha_values'])
n_sigma = len(SWEEP_CONFIG['sigma_values'])
n_combos = n_alpha * n_sigma
total_sims = n_combos * SWEEP_CONFIG['n_runs_per_combo']

print("=" * 60)
print("EXPERIMENT 10b: 2D ALPHA-SIGMA SWEEP CONFIGURATION")
print("=" * 60)
print(f"Alpha values: {SWEEP_CONFIG['alpha_values']}")
print(f"Sigma values: {SWEEP_CONFIG['sigma_values']}")
print(f"Grid size: {n_alpha} x {n_sigma} = {n_combos} combinations")
print(f"Runs per combination: {SWEEP_CONFIG['n_runs_per_combo']}")
print(f"Total simulations: {total_sims}")
print(f"\nEstimated runtime: ~{total_sims * 67 / 300 / 60:.1f} hours (based on Exp 10)")
print(f"\nFixed parameters:")
print(f"  Cascade alpha: {SWEEP_CONFIG['cascade_alpha']}")
print(f"  Cascade sigma: {SWEEP_CONFIG['cascade_sigma']}")
print(f"  Barrier height: {SWEEP_CONFIG['barrier_height']}")

## 4. Network Creation

In [ ]:
def create_sweep_network(data, config, seed=42):
    """
    Create a 50-cell network for sweep experiments.
    Uses symmetric EnergyConstrainedCusp elements.
    """
    np.random.seed(seed)
    
    network_matrix = data['network']
    n_cells = config['n_cells']
    min_flow = config['min_flow']
    barrier_height = config['barrier_height']
    
    # Select top cells by total connectivity
    total_flow = network_matrix.sum(axis=0) + network_matrix.sum(axis=1)
    top_indices = np.argsort(total_flow)[-n_cells:]
    
    # Create network
    net = EnergyConstrainedNetwork()
    
    # Add cusp elements
    for i, idx in enumerate(top_indices):
        element = EnergyConstrainedCusp(
            a=-1.0, b=1.0, c=0.0, x_0=0.0,
            barrier_height=barrier_height,
            dissipation_rate=0.1
        )
        net.add_element(f'cell_{i}', element)
    
    # Add couplings based on moisture flow
    n_edges = 0
    for i, idx_i in enumerate(top_indices):
        for j, idx_j in enumerate(top_indices):
            if i != j:
                flow = network_matrix[idx_i, idx_j]
                if flow > min_flow:
                    coupling = GradientDrivenCoupling(
                        conductivity=flow / 100.0,
                        state_coupling=0.1
                    )
                    net.add_coupling(f'cell_{i}', f'cell_{j}', coupling)
                    n_edges += 1
    
    return net, top_indices

# Create the network
network, selected_cells = create_sweep_network(amazon_data, SWEEP_CONFIG)
print(f"Created network: {network.n_elements} nodes, {network.number_of_edges()} edges")

## 5. Worker Function (Optimized)

In [ ]:
def run_single_experiment(args):
    """
    Worker function for a single (alpha, sigma) experiment.
    
    Parameters
    ----------
    args : tuple
        (network_bytes, recovery_alpha, recovery_sigma, config, seed)
    
    Returns
    -------
    dict
        Results dictionary for DataFrame aggregation
    """
    network_bytes, recovery_alpha, recovery_sigma, config, seed = args
    
    import sys
    import numpy as np
    import pickle
    
    # Add path for k3s workers
    if '/opt/research-local/src' not in sys.path:
        sys.path.insert(0, '/opt/research-local/src')
    
    from energy_constrained.solvers import run_two_phase_experiment
    
    # Reconstruct network
    network = pickle.loads(network_bytes)
    
    # Set seed
    np.random.seed(seed)
    
    # Run two-phase experiment
    result = run_two_phase_experiment(
        network=network,
        cascade_duration=config['cascade_duration'],
        recovery_duration=config['recovery_duration'],
        dt=config['dt'],
        cascade_sigma=config['cascade_sigma'],
        cascade_alpha=config['cascade_alpha'],
        recovery_sigma=recovery_sigma,  # SWEEP VARIABLE
        recovery_alpha=recovery_alpha,  # SWEEP VARIABLE
        seed=seed
    )
    
    # Compute additional metrics
    pct_time_tipped = np.mean(result.x_full > 0) * 100
    
    # Count transitions
    n_cells = result.x_full.shape[1]
    n_tip_events = 0
    n_recover_events = 0
    
    for j in range(n_cells):
        x_traj = result.x_full[:, j]
        signs = np.sign(x_traj)
        sign_changes = np.diff(signs)
        n_tip_events += np.sum(sign_changes > 0)
        n_recover_events += np.sum(sign_changes < 0)
    
    return {
        'recovery_alpha': recovery_alpha,
        'recovery_sigma': recovery_sigma,
        'seed': seed,
        'recovery_fraction': result.metrics['recovery_fraction'],
        'pct_tipped_cascade': result.metrics['pct_tipped_at_cascade_end'],
        'n_permanent_tips': result.metrics['n_permanent_tips'],
        'final_pct_tipped': result.metrics['final_pct_tipped'],
        'pct_time_tipped': pct_time_tipped,
        'n_tip_events': n_tip_events,
        'n_recover_events': n_recover_events,
        'mean_recovery_time': result.metrics['mean_recovery_time'],
    }

print("Worker function defined.")

## 6. Run 2D Sweep Experiment

In [ ]:
# Serialize network once
network_bytes = pickle.dumps(network)
print(f"Network serialized: {len(network_bytes) / 1024:.1f} KB")

# Pre-scatter network to all workers for efficiency
network_future = client.scatter(network_bytes, broadcast=True)
print("Network broadcast to all workers")

# Build task arguments
print("\n" + "=" * 60)
print("EXPERIMENT 10b: Starting 2D Alpha-Sigma Sweep")
print("=" * 60)
start_time = time.time()

# Generate all (alpha, sigma, run) combinations
task_args = []
combo_idx = 0
for alpha in SWEEP_CONFIG['alpha_values']:
    for sigma in SWEEP_CONFIG['sigma_values']:
        for run_idx in range(SWEEP_CONFIG['n_runs_per_combo']):
            seed = SWEEP_CONFIG['base_seed'] + combo_idx * 1000 + run_idx
            task_args.append((network_bytes, float(alpha), float(sigma), SWEEP_CONFIG, seed))
        combo_idx += 1

print(f"Generated {len(task_args)} task arguments")
print(f"Parameter grid: {n_alpha} alphas x {n_sigma} sigmas x {SWEEP_CONFIG['n_runs_per_combo']} runs")

# Submit using client.map for efficient batching
futures = client.map(run_single_experiment, task_args)
print(f"Submitted {len(futures)} tasks via client.map()")

# Collect results with progress tracking
all_results = []
print("\nProgress:")

for i, future in enumerate(as_completed(futures)):
    result = future.result()
    all_results.append(result)
    
    # Print progress every 50 completions
    if (i + 1) % 50 == 0:
        elapsed = time.time() - start_time
        rate = (i + 1) / elapsed
        remaining = (len(futures) - i - 1) / rate if rate > 0 else 0
        print(f"  Completed {i+1}/{len(futures)} ({100*(i+1)/len(futures):.1f}%) - "
              f"Elapsed: {elapsed/60:.1f}min, ETA: {remaining/60:.1f}min")

elapsed = time.time() - start_time
print(f"\n" + "=" * 60)
print(f"COMPLETE: {len(all_results)} simulations in {elapsed:.1f}s ({elapsed/60:.1f} min)")
print(f"Average per simulation: {elapsed/len(all_results):.2f} seconds")
print("=" * 60)

## 7. Results Aggregation

In [ ]:
# Convert to DataFrame
df = pd.DataFrame(all_results)
print(f"Results shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

# Sort for consistent ordering
df = df.sort_values(['recovery_alpha', 'recovery_sigma', 'seed']).reset_index(drop=True)

df.head(10)

In [ ]:
# Summary statistics by (alpha, sigma) combination
summary = df.groupby(['recovery_alpha', 'recovery_sigma']).agg({
    'recovery_fraction': ['mean', 'std'],
    'n_permanent_tips': ['mean', 'std'],
    'final_pct_tipped': ['mean', 'std'],
    'n_recover_events': ['mean', 'std'],
}).round(4)

summary.columns = ['_'.join(col) for col in summary.columns]
summary = summary.reset_index()

print("=" * 80)
print("2D SWEEP SUMMARY: Recovery by (Alpha, Sigma)")
print("=" * 80)
print(summary[['recovery_alpha', 'recovery_sigma', 'recovery_fraction_mean', 
               'recovery_fraction_std', 'n_permanent_tips_mean']].to_string(index=False))

## 8. Primary Visualization - Recovery Heatmap

In [ ]:
# Create pivot table for heatmap
pivot_recovery = summary.pivot(index='recovery_alpha', columns='recovery_sigma', 
                                values='recovery_fraction_mean')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Panel 1: Recovery Fraction Heatmap
ax = axes[0]
im = ax.imshow(pivot_recovery.values, cmap='RdYlGn', aspect='auto', 
               vmin=0, vmax=max(0.5, pivot_recovery.values.max()))
ax.set_xticks(range(len(SWEEP_CONFIG['sigma_values'])))
ax.set_xticklabels([f'{s:.2f}' for s in SWEEP_CONFIG['sigma_values']])
ax.set_yticks(range(len(SWEEP_CONFIG['alpha_values'])))
ax.set_yticklabels([f'{a:.1f}' for a in SWEEP_CONFIG['alpha_values']])
ax.set_xlabel('Recovery σ (noise amplitude)', fontsize=12)
ax.set_ylabel('Recovery α (Lévy stability)', fontsize=12)
ax.set_title('Recovery Fraction by (α, σ)', fontsize=14)

# Add values to cells
for i in range(len(SWEEP_CONFIG['alpha_values'])):
    for j in range(len(SWEEP_CONFIG['sigma_values'])):
        val = pivot_recovery.values[i, j]
        color = 'white' if val < 0.25 else 'black'
        ax.text(j, i, f'{val:.2f}', ha='center', va='center', color=color, fontsize=10)

plt.colorbar(im, ax=ax, label='Recovery Fraction')

# Panel 2: Permanent Tips Heatmap
pivot_tips = summary.pivot(index='recovery_alpha', columns='recovery_sigma',
                           values='n_permanent_tips_mean')

ax = axes[1]
im2 = ax.imshow(pivot_tips.values, cmap='YlOrRd', aspect='auto')
ax.set_xticks(range(len(SWEEP_CONFIG['sigma_values'])))
ax.set_xticklabels([f'{s:.2f}' for s in SWEEP_CONFIG['sigma_values']])
ax.set_yticks(range(len(SWEEP_CONFIG['alpha_values'])))
ax.set_yticklabels([f'{a:.1f}' for a in SWEEP_CONFIG['alpha_values']])
ax.set_xlabel('Recovery σ (noise amplitude)', fontsize=12)
ax.set_ylabel('Recovery α (Lévy stability)', fontsize=12)
ax.set_title('Permanent Tips by (α, σ)', fontsize=14)

# Add values to cells
for i in range(len(SWEEP_CONFIG['alpha_values'])):
    for j in range(len(SWEEP_CONFIG['sigma_values'])):
        val = pivot_tips.values[i, j]
        color = 'white' if val > 30 else 'black'
        ax.text(j, i, f'{val:.1f}', ha='center', va='center', color=color, fontsize=10)

plt.colorbar(im2, ax=ax, label='Permanent Tips (out of 50)')

plt.tight_layout()
plt.savefig('/workspace/data/exp10b_heatmaps.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nPlot saved to /workspace/data/exp10b_heatmaps.png")

## 9. Line Plots - Recovery vs σ at Each α

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = plt.cm.viridis(np.linspace(0, 1, len(SWEEP_CONFIG['alpha_values'])))

# Panel 1: Recovery vs Sigma, lines for each Alpha
ax = axes[0]
for i, alpha in enumerate(SWEEP_CONFIG['alpha_values']):
    subset = summary[summary['recovery_alpha'] == alpha]
    ax.errorbar(subset['recovery_sigma'], subset['recovery_fraction_mean'],
                yerr=subset['recovery_fraction_std'], marker='o', capsize=3,
                label=f'α={alpha:.1f}', color=colors[i], linewidth=2, markersize=8)

ax.axhline(0.1, color='red', linestyle='--', alpha=0.5, label='10% threshold')
ax.axhline(0.3, color='orange', linestyle='--', alpha=0.5, label='30% threshold')
ax.set_xlabel('Recovery σ', fontsize=12)
ax.set_ylabel('Recovery Fraction', fontsize=12)
ax.set_title('Recovery vs Noise Amplitude by α', fontsize=14)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)
ax.set_ylim(0, max(0.6, summary['recovery_fraction_mean'].max() + 0.1))

# Panel 2: Recovery vs Alpha, lines for each Sigma
ax = axes[1]
colors2 = plt.cm.plasma(np.linspace(0, 1, len(SWEEP_CONFIG['sigma_values'])))
for i, sigma in enumerate(SWEEP_CONFIG['sigma_values']):
    subset = summary[summary['recovery_sigma'] == sigma]
    ax.errorbar(subset['recovery_alpha'], subset['recovery_fraction_mean'],
                yerr=subset['recovery_fraction_std'], marker='s', capsize=3,
                label=f'σ={sigma:.2f}', color=colors2[i], linewidth=2, markersize=8)

ax.axhline(0.1, color='red', linestyle='--', alpha=0.5, label='10% threshold')
ax.axhline(0.3, color='orange', linestyle='--', alpha=0.5, label='30% threshold')
ax.set_xlabel('Recovery α', fontsize=12)
ax.set_ylabel('Recovery Fraction', fontsize=12)
ax.set_title('Recovery vs Lévy Stability by σ', fontsize=14)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
ax.set_ylim(0, max(0.6, summary['recovery_fraction_mean'].max() + 0.1))

plt.tight_layout()
plt.savefig('/workspace/data/exp10b_lines.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nPlot saved to /workspace/data/exp10b_lines.png")

## 10. Critical Threshold Analysis

In [ ]:
def find_sigma_threshold(summary_df, alpha, recovery_threshold=0.1):
    """
    Find minimum sigma needed to achieve recovery_threshold at given alpha.
    Uses linear interpolation between grid points.
    """
    subset = summary_df[summary_df['recovery_alpha'] == alpha].sort_values('recovery_sigma')
    sigmas = subset['recovery_sigma'].values
    recoveries = subset['recovery_fraction_mean'].values
    
    # Find crossing point
    for i in range(len(sigmas) - 1):
        if recoveries[i] < recovery_threshold <= recoveries[i+1]:
            # Linear interpolation
            slope = (recoveries[i+1] - recoveries[i]) / (sigmas[i+1] - sigmas[i])
            if slope > 0:
                sigma_critical = sigmas[i] + (recovery_threshold - recoveries[i]) / slope
                return sigma_critical
    
    # Check if always above or below
    if recoveries.max() < recovery_threshold:
        return None  # Never reaches threshold
    elif recoveries.min() >= recovery_threshold:
        return sigmas.min()  # Always above threshold
    
    return None


print("=" * 60)
print("CRITICAL SIGMA THRESHOLDS")
print("=" * 60)

threshold_results = []
for recovery_thresh in [0.10, 0.20, 0.30]:
    print(f"\nRecovery threshold: {recovery_thresh*100:.0f}%")
    for alpha in SWEEP_CONFIG['alpha_values']:
        sigma_crit = find_sigma_threshold(summary, alpha, recovery_thresh)
        if sigma_crit is not None:
            print(f"  α={alpha:.1f}: σ_critical = {sigma_crit:.3f}")
            threshold_results.append({
                'recovery_threshold': recovery_thresh,
                'alpha': alpha,
                'sigma_critical': sigma_crit
            })
        else:
            print(f"  α={alpha:.1f}: threshold not reached in σ range")

threshold_df = pd.DataFrame(threshold_results)
if len(threshold_df) > 0:
    print("\nCritical thresholds found:")
    print(threshold_df.to_string(index=False))

## 11. Regime Classification

In [ ]:
def classify_regime(recovery_fraction):
    if recovery_fraction >= 0.30:
        return 'Recovery-capable'
    elif recovery_fraction >= 0.10:
        return 'Transition'
    else:
        return 'Trapped'

summary['regime'] = summary['recovery_fraction_mean'].apply(classify_regime)

# Create regime heatmap
regime_map = {'Trapped': 0, 'Transition': 1, 'Recovery-capable': 2}
summary['regime_code'] = summary['regime'].map(regime_map)

pivot_regime = summary.pivot(index='recovery_alpha', columns='recovery_sigma',
                              values='regime_code')

fig, ax = plt.subplots(figsize=(10, 6))

# Custom colormap: red (trapped) -> yellow (transition) -> green (recovery)
from matplotlib.colors import ListedColormap
cmap = ListedColormap(['#d73027', '#fee08b', '#1a9850'])

im = ax.imshow(pivot_regime.values, cmap=cmap, aspect='auto', vmin=0, vmax=2)
ax.set_xticks(range(len(SWEEP_CONFIG['sigma_values'])))
ax.set_xticklabels([f'{s:.2f}' for s in SWEEP_CONFIG['sigma_values']])
ax.set_yticks(range(len(SWEEP_CONFIG['alpha_values'])))
ax.set_yticklabels([f'{a:.1f}' for a in SWEEP_CONFIG['alpha_values']])
ax.set_xlabel('Recovery σ (noise amplitude)', fontsize=12)
ax.set_ylabel('Recovery α (Lévy stability)', fontsize=12)
ax.set_title('Regime Classification Map', fontsize=14)

# Add regime labels to cells
regime_labels = {0: 'T', 1: 'TR', 2: 'R'}
for i in range(len(SWEEP_CONFIG['alpha_values'])):
    for j in range(len(SWEEP_CONFIG['sigma_values'])):
        val = pivot_regime.values[i, j]
        label = regime_labels.get(val, '?')
        color = 'white' if val == 0 else 'black'
        ax.text(j, i, label, ha='center', va='center', color=color, fontsize=12, fontweight='bold')

# Legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#d73027', label='Trapped (T): <10% recovery'),
    Patch(facecolor='#fee08b', label='Transition (TR): 10-30% recovery'),
    Patch(facecolor='#1a9850', label='Recovery-capable (R): >30% recovery')
]
ax.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1.02, 1))

plt.tight_layout()
plt.savefig('/workspace/data/exp10b_regime_map.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nPlot saved to /workspace/data/exp10b_regime_map.png")

## 12. Key Findings Summary

In [ ]:
print("\n" + "=" * 70)
print("EXPERIMENT 10b: KEY FINDINGS")
print("=" * 70)

# Count regimes
regime_counts = summary['regime'].value_counts()

# Find best and worst combinations
best_combo = summary.loc[summary['recovery_fraction_mean'].idxmax()]
worst_combo = summary.loc[summary['recovery_fraction_mean'].idxmin()]

print(f"""
1. PARAMETER SPACE COVERAGE:
   Grid: {n_alpha} α values × {n_sigma} σ values = {n_combos} combinations
   Runs per combination: {SWEEP_CONFIG['n_runs_per_combo']}
   Total simulations: {len(df)}

2. REGIME DISTRIBUTION:
   Trapped (<10% recovery):        {regime_counts.get('Trapped', 0)} combinations
   Transition (10-30% recovery):   {regime_counts.get('Transition', 0)} combinations  
   Recovery-capable (>30%):        {regime_counts.get('Recovery-capable', 0)} combinations

3. BEST RECOVERY:
   α={best_combo['recovery_alpha']:.1f}, σ={best_combo['recovery_sigma']:.2f}
   Recovery fraction: {best_combo['recovery_fraction_mean']:.3f} ± {best_combo['recovery_fraction_std']:.3f}

4. WORST RECOVERY:
   α={worst_combo['recovery_alpha']:.1f}, σ={worst_combo['recovery_sigma']:.2f}
   Recovery fraction: {worst_combo['recovery_fraction_mean']:.3f} ± {worst_combo['recovery_fraction_std']:.3f}

5. KEY INSIGHTS:
""")

# Analyze which parameter matters more
alpha_effect = summary.groupby('recovery_alpha')['recovery_fraction_mean'].mean()
sigma_effect = summary.groupby('recovery_sigma')['recovery_fraction_mean'].mean()

alpha_range = alpha_effect.max() - alpha_effect.min()
sigma_range = sigma_effect.max() - sigma_effect.min()

print(f"   Effect of α (averaging over σ): range = {alpha_range:.3f}")
print(f"   Effect of σ (averaging over α): range = {sigma_range:.3f}")

if sigma_range > alpha_range * 1.5:
    print("   → σ (noise amplitude) has STRONGER effect than α (noise type)")
elif alpha_range > sigma_range * 1.5:
    print("   → α (noise type) has STRONGER effect than σ (noise amplitude)")
else:
    print("   → Both parameters have comparable effects")

# Check for interaction
print(f"\n6. α × σ INTERACTION:")
for alpha in [SWEEP_CONFIG['alpha_values'].min(), SWEEP_CONFIG['alpha_values'].max()]:
    subset = summary[summary['recovery_alpha'] == alpha]
    slope = (subset['recovery_fraction_mean'].iloc[-1] - subset['recovery_fraction_mean'].iloc[0]) / \
            (subset['recovery_sigma'].iloc[-1] - subset['recovery_sigma'].iloc[0])
    print(f"   At α={alpha:.1f}: d(recovery)/d(σ) ≈ {slope:.2f}")

## 13. Save Results

In [ ]:
# Save full results
df.to_csv('/workspace/data/experiment10b_alpha_sigma_results.csv', index=False)
print(f"Full results saved to /workspace/data/experiment10b_alpha_sigma_results.csv")

# Save summary
summary.to_csv('/workspace/data/experiment10b_summary.csv', index=False)
print(f"Summary saved to /workspace/data/experiment10b_summary.csv")

# Save threshold analysis
if len(threshold_df) > 0:
    threshold_df.to_csv('/workspace/data/experiment10b_thresholds.csv', index=False)
    print(f"Thresholds saved to /workspace/data/experiment10b_thresholds.csv")

In [ ]:
print("\n" + "=" * 70)
print("EXPERIMENT 10b COMPLETE")
print("=" * 70)

print(f"""
CONFIGURATION:
- Alpha values: {list(SWEEP_CONFIG['alpha_values'])}
- Sigma values: {list(SWEEP_CONFIG['sigma_values'])}
- Grid: {n_combos} combinations
- Runs per combo: {SWEEP_CONFIG['n_runs_per_combo']}
- Total simulations: {len(df)}
- Runtime: {elapsed:.1f}s ({elapsed/60:.1f} min)

FILES GENERATED:
- /workspace/data/experiment10b_alpha_sigma_results.csv
- /workspace/data/experiment10b_summary.csv
- /workspace/data/experiment10b_thresholds.csv
- /workspace/data/exp10b_heatmaps.png
- /workspace/data/exp10b_lines.png
- /workspace/data/exp10b_regime_map.png

NEXT STEPS:
1. Update docs/phase4_results.md with findings
2. If recovery region found, proceed to Experiment 11 (Keystone Connections)
3. If still no recovery, consider reducing barrier height further
""")